In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(2)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1122,tt0322330,26000000,110230332,Freaky Friday,Jamie Lee Curtis|Lindsay Lohan|Mark Harmon|Har...,Mark Waters,Mondays are manic. Wednesdays are wild. And Fr...,Mother and daughter bicker over everything -- ...,97,Comedy|Family|Fantasy,Walt Disney Pictures,8/3/2003,6.0,2003
639,tt0758774,70000000,113280098,Body of Lies,Leonardo DiCaprio|Russell Crowe|Mark Strong|Al...,Ridley Scott,Trust no one. Deceive everyone.,"Body of Lies follows CIA operative, Roger Ferr...",128,Action|Drama|Thriller,Scott Free Productions|De Line Pictures|Warner...,10/10/2008,6.5,2008


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.describe(include = object)

,imdb_id,original_title,cast,director,tagline,overview,genres,production_companies,release_date
count,1889,1889,1889,1889,1889,1889,1889,1889,1889
unique,1889,1887,1878,957,1880,1889,652,1624,1435
top,tt0427309,Fantastic Four,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Steven Soderbergh,The only way out is down.,"Dr. John Dolittle the beloved doctor is back, ...",Comedy,New Line Cinema,12/25/2014
freq,1,2,3,13,2,1,141,21,7


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

 1. В DataFrame data добавлен столбец "profit" как разница доходов и бюджета;
 
 
 2. Создан DataFrame data_genres с двумя столбцами:
         'imdb_id' - универсальный код фильма на imdb;
         'genres_code' - кодировка жанров в виде массива 0-й и 1-ц.
         
 3. Создан DataFrame data_genres_parsed с n+1 столбцами:
         n - число возможных жанров;
         Число 1 в клетке означает соответствие фильма жанру, а 0 - противное.
         
 4. Создан DataFrame data_dates с 4-мя столбцами:
         'imdb_id' - универсальный код фильма на imdb;
         'day', 'month', 'year' - днём, месяцем и годом даты релиза фильма.
         
 5. Создан data_actors c 2-мя столбцами:
         'imdb_id' - универсальный код фильма на imdb;
         'actor' - имя и фамилия актёра из фильма.
Так как для одного фильма может быть несколько актёров, то это привело к увеличению числа строк. Для склейки будем пользоваться 'imdb_id'.
         
         
 6. Создан data_directors c 2-мя столбцами:
         'imdb_id' - универсальный код фильма на imdb;
         'director' - имя и фамилия директора (режиссёра) фильма.
Так как для одного фильма может быть несколько директоров (режиссёров), то это привело к увеличению числа строк. Для склейки будем пользоваться 'imdb_id'.
         
         
 7. Создан data_companies c 2-мя столбцами:
         'imdb_id' - универсальный код фильма на imdb;
         'production_company' - названии студии, участвующей в съёмках фильма.      
Так как для одного фильма может быть несколько студий, то это привело к увеличению числа строк. Для склейки будем пользоваться 'imdb_id'.

In [6]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

## 1. DataFrame *data*.

In [7]:
data['profit'] = data.revenue - data.budget

## 2. DataFrame *data_genres*.

In [8]:
#genres - list of the all genres listed in data DataFrame.

genres = []
for _tmp in data.genres.unique():
    for genre in _tmp.split('|'):
        genres.append(genre)
genres = list(set(genres))


# genre_encoding function consumes the string of genres seperated with symbol '|' 
# and returns the numpy array of zeros and ones,
# where one means the genre in genres at the same position.

def genre_encoding(movie_genres):
    genres_code = np.zeros(len(genres))
    for i, genre in  enumerate(genres):
        if genre in movie_genres.split('|'):
            genres_code[i] = 1
    return genres_code


# genre_decoding function consumes the list / numpy array of zeros and ones,
# where one means the genre in genres at the same position,
# and returns the string of genres seperated with symbol '|' 

def genre_decoding(genres_code):
    movie_genres = ''
    for i, code in enumerate(genres_code):
        if code != 0.0:
            movie_genres += genres[i] + '|'
    return movie_genres[:-1]


data_genres_columns = ['imdb_id', 'genres_code']
data_genres = pd.DataFrame(columns=data_genres_columns)
data_genres['imdb_id'] = data['imdb_id']
data_genres['genres_code'] = data['genres'].apply(genre_encoding)
data['genres_code'] = data['genres'].apply(genre_encoding)

display(data_genres.iloc[0])
display(data['genres'].iloc[0])

imdb_id                                                tt0369610
genres_code    [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: 0, dtype: object

'Action|Adventure|Science Fiction|Thriller'

## 3. DataFrame *data_genres_parsed*.

In [9]:
data_genres_parsed = data_genres.copy(deep=True)
data_genres_parsed[genres] = pd.DataFrame(data_genres['genres_code'].tolist(),
                                          index=data_genres.index)
data_genres_parsed = data_genres_parsed.drop('genres_code', axis=1) 

#Print the 1st row to check the parse
display(data_genres_parsed.iloc[0])
display(data['genres'].iloc[0])

imdb_id            tt0369610
Music                      0
Thriller                   1
Adventure                  1
War                        0
Comedy                     0
Family                     0
Foreign                    0
Western                    0
Documentary                0
Science Fiction            1
Crime                      0
Action                     1
Drama                      0
Romance                    0
Horror                     0
History                    0
Animation                  0
Fantasy                    0
Mystery                    0
Name: 0, dtype: object

'Action|Adventure|Science Fiction|Thriller'

## 4. DataFrame *data_dates*.

In [10]:
data_dates = pd.DataFrame(data['release_date'].str.split('/').tolist(),
                          columns=['month', 'day', 'year'])

data_dates.insert(loc=0, column='imdb_id', value=data['imdb_id'])

for column_name in ['month', 'day', 'year']:
    data_dates[column_name] = data_dates[column_name].apply(lambda x: int(x))

display(data_dates.sample(5))


# Check the year in release_data and release_year in data.

print((data_dates['year'] == data['release_year']).any())

,imdb_id,month,day,year
799,tt1152398,1,21,2011
1730,tt0481141,7,25,2007
359,tt0458525,4,28,2009
1034,tt1430626,3,12,2012
1116,tt0302886,2,21,2003


True


## 5. DataFrame *data_actors*.

In [11]:
data_actors = data[['imdb_id','cast']].copy(deep=True)
try:
    data_actors = data_actors.rename(columns={'cast':'actor'})
    data_actors['actor'] = data_actors['actor'].str.split('|')
    data_actors = data_actors.explode('actor')
    data_actors = data_actors.reset_index(drop=True)
except:
    pass
display(data_actors.head(5))
display(data['cast'].iloc[0])

,imdb_id,actor
0,tt0369610,Chris Pratt
1,tt0369610,Bryce Dallas Howard
2,tt0369610,Irrfan Khan
3,tt0369610,Vincent D'Onofrio
4,tt0369610,Nick Robinson


"Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vincent D'Onofrio|Nick Robinson"

## 5. DataFrame *data_directors*.

In [12]:
data_directors = data[['imdb_id','director']].copy(deep=True)

try:
    data_directors = data_directors.rename(columns={'director':'author'})
    data_directors['author'] = data_directors['author'].str.split('|')
    data_directors = data_directors.explode('author')
    data_directors = data_directors.reset_index(drop=True)
except:
    pass

display(data_directors.head(5))

,imdb_id,author
0,tt0369610,Colin Trevorrow
1,tt1392190,George Miller
2,tt2908446,Robert Schwentke
3,tt2488496,J.J. Abrams
4,tt2820852,James Wan


## 6. DataFrame data_companies

In [13]:
data_companies = data[['imdb_id','production_companies']].copy(deep=True)

try:
    data_companies = data_companies.rename(columns={'production_companies':'company'})
    data_companies['company'] = data_companies['company'].str.split('|')
    data_companies = data_companies.explode('company')
    data_companies = data_companies.reset_index(drop=True)
except:
    pass

display(data_companies.head(5))
display(data['production_companies'].iloc[0])

,imdb_id,company
0,tt0369610,Universal Studios
1,tt0369610,Amblin Entertainment
2,tt0369610,Legendary Pictures
3,tt0369610,Fuji Television Network
4,tt0369610,Dentsu


'Universal Studios|Amblin Entertainment|Legendary Pictures|Fuji Television Network|Dentsu'

## 7. Additional notation

Next notations are used denote some temprorary parameters:
    - data_XX_YY is DataFrame, index 'XX' is question number and sunindex 'YY' is number of DataFrame used during the question performance. If YY equals to zero, this subindex is suppresed.
    
    - pivot_XX_YY is PivotTable, index 'XX' is question number and sunindex 'YY' is number of DataFrame used during the question performance. If YY equals to zero, this subindex is suppresed.
    
    - answer_XX is answer storage parameter, where 'XX' is question number. Used to clear the code.

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [14]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [15]:
movie_budget_max = data[data.budget == data.budget.max()]
answer_1 = movie_budget_max[['imdb_id', 'original_title']]


print('1. Самый большой бюджет у фильма из списка у фильма:', answer_1['original_title'].iloc[0])

1. Самый большой бюджет у фильма из списка у фильма: Pirates of the Caribbean: On Stranger Tides


# 2. Какой из фильмов самый длительный (в минутах)?

In [16]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [17]:
movie_runtime_max = data[data.runtime == data.runtime.max()]
answer_2 = movie_runtime_max[['imdb_id', 'original_title']]


print('2. Самый длинный фильм:', answer_2['original_title'].iloc[0])

2. Самый длинный фильм: Gods and Generals


# 3. Какой из фильмов самый короткий (в минутах)?

In [18]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [19]:
movie_runtime_min = data[data.runtime == data.runtime.min()]
answer_3 = movie_runtime_min[['imdb_id', 'original_title']]


print('3. Самый короткий фильм:', answer_3['original_title'].iloc[0])

3. Самый короткий фильм: Winnie the Pooh


# 4. Какова средняя длительность фильмов?


In [20]:
answers['4'] = '2. 110'

In [21]:
answer_4 = round(data.runtime.mean(),0)


print('4. Средняя длительность фильмов: ', int(answer_4), 'минут')

4. Средняя длительность фильмов:  110 минут


# 5. Каково медианное значение длительности фильмов? 

In [22]:
answers['5'] = '2. 107'

In [23]:
answer_5 = round(data.runtime.median(),0)


print('4. Медианная длительность фильмов: ', int(answer_5), 'минут')

4. Медианная длительность фильмов:  107 минут


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [24]:
answers['6'] = '5. Avatar (tt0499549)'

In [25]:
movie_profit_max = data[data.profit == data.profit.max()]
answer_6 = movie_profit_max[['imdb_id', 'original_title']]


print('6. Самый прибыльный фильм:', answer_6['original_title'].iloc[0])

6. Самый прибыльный фильм: Avatar


# 7. Какой фильм самый убыточный? 

In [26]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [27]:
movie_profit_min = data[data.profit == data.profit.min()]
answer_7 = movie_profit_min[['imdb_id', 'original_title']]


print('7. Самый убыточный фильм:', answer_7['original_title'].iloc[0])

7. Самый убыточный фильм: The Lone Ranger


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [28]:
answers['8'] = '1. 1478'

In [29]:
answer_8 = len(data[data.profit>0])


print('8. У', answer_8, 'фильмов из датасета объём сборов больше бюджета.')

8. У 1478 фильмов из датасета объём сборов больше бюджета.


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [30]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [31]:
data_9 = data[data.release_year == 2008]
answer_9 = data_9[data_9.profit == data_9.profit.max()]


print('9. Самый кассовый фильм 2008 года:', answer_9['original_title'].iloc[0])

9. Самый кассовый фильм 2008 года: The Dark Knight


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [32]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

ВАРИАНТ 1

In [33]:
data_10_1 = data[data.release_year == 2012]
movie_profit_min_2012 = data_10_1[data_10_1.profit == data_10_1.profit.min()]

data_10_2 = data[data.release_year == 2013]
movie_profit_min_2013 = data_10_2[data_10_2.profit == data_10_2.profit.min()]

data_10_3 = data[data.release_year == 2014]
movie_profit_min_2014 = data_10_3[data_10_3.profit == data_10_3.profit.min()]

data_10_4 = pd.concat([movie_profit_min_2012,movie_profit_min_2013, movie_profit_min_2014])
answer_10 = data_10_4[data_10_4.profit == data_10_4.profit.min()]


print('10. Самый убыточный фильм за период с 2012 по 2014:', answer_10['original_title'].iloc[0])

10. Самый убыточный фильм за период с 2012 по 2014: The Lone Ranger


ВАРИАНТ 2

In [34]:
data_10 = data[(data.release_year == 2012) | (data.release_year == 2013) | (data.release_year == 2014)]
answer_10 = data_10[data_10.profit == data_10.profit.min()]


print('10. Самый убыточный фильм за период с 2012 по 2014:', answer_10['original_title'].iloc[0])

10. Самый убыточный фильм за период с 2012 по 2014: The Lone Ranger


ВАРИАНТ 3

In [35]:
data_10 = data[data['release_year'].isin([2012,2013,2014])]
answer_10 = data_10[data_10.profit == data_10.profit.min()]


print('10. Самый убыточный фильм за период с 2012 по 2014:', answer_10['original_title'].iloc[0])

10. Самый убыточный фильм за период с 2012 по 2014: The Lone Ranger


# 11. Какого жанра фильмов больше всего?

In [36]:
answers['11'] = '3. Drama'

ВАРИАНТ 1

In [37]:
genre_max = np.zeros(len(genres))
genre_max[np.argmax(data_genres.genres_code.sum())] = 1.0


print('11. Больше всего фильмов в жанре: ', genre_decoding(genre_max))

11. Больше всего фильмов в жанре:  Drama


ВАРИАНТ 2

In [38]:
answer_11 = data_genres_parsed.drop('imdb_id', axis=1).sum().sort_values(ascending=False).head(1)


print('11. Больше всего фильмов в жанре: ', answer_11.index[0])

11. Больше всего фильмов в жанре:  Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [39]:
answers['12'] = '1. Drama'

In [40]:
answer_12 = genres[np.argmax(data[data.profit > 0].genres_code.sum())]


print('12. Чаще всего становятся прибыльными фильмы в жанре:', answer_12)

12. Чаще всего становятся прибыльными фильмы в жанре: Drama


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [41]:
answers['13'] = '5. Peter Jackson'

ВАРИАНТ 1

In [42]:
data_13 = data_directors.merge(data, how='outer', on='imdb_id')
pivot_13 = data_13.pivot_table(index='author', values='revenue',
                               aggfunc='sum', fill_value=0)
answer_13 = pivot_13.sort_values(by='revenue', ascending=False).iloc[0]


print('13. Самый большие суммарные кассовые сборы у режиссёра:', answer_13.name)

13. Самый большие суммарные кассовые сборы у режиссёра: Peter Jackson


ВАРИАНТ 2

In [43]:
# Hypothesis: there could be movies shot by several directors, but they will not be the top 1.

director_revenue_sum = data.groupby(['director'])['revenue'].sum().sort_values(ascending=False)
answer_13 = director_revenue_sum.head(1).index[0]


print('13. Самый большие суммарные кассовые сборы у режиссёра:', answer_13)

13. Самый большие суммарные кассовые сборы у режиссёра: Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [44]:
answers['14'] = '3. Robert Rodriguez'

In [45]:
data_14 = data_directors.merge(data_genres_parsed, how='outer', on='imdb_id')
pivot_14 = data_14.pivot_table(index='author', values='Action',
                               aggfunc='sum', fill_value=0)
answer_14 = pivot_14.sort_values(by='Action',ascending=False).iloc[0]


print('14. Больше всего фильмов в жанре Action снял:', answer_14.name)

14. Больше всего фильмов в жанре Action снял: Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [46]:
answers['15'] = '3. Chris Hemsworth'

In [47]:
data_15 = data[data['release_year'] == 2012]
data_15 = data_15.merge(data_actors, how='inner', on='imdb_id')
data_15 = data_15[['imdb_id', 'revenue', 'release_year', 'actor']]
pivot_15 = data_15.pivot_table(index='actor', values='revenue',
                               aggfunc='sum', fill_value=0)
answer_15 = pivot_15.sort_values(by='revenue',ascending=False).iloc[0]


print('15. Самые высокие кассовые сборы в 2012 году принесли фильмы с участием:', answer_15.name)

15. Самые высокие кассовые сборы в 2012 году принесли фильмы с участием: Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?
Примечание: в фильмах, где бюджет выше среднего по данной выборке.

In [48]:
answers['16'] = '3. Matt Damon'

In [49]:
data_16 = data[data['budget'] > data['budget'].mean()]
data_16 = data_16.merge(data_actors, how='inner', on='imdb_id')
answer_16 = data_16['actor'].value_counts().head(1).index[0]


print('16. В большем количестве высокобюджетных фильмов снялся:', answer_16)

16. В большем количестве высокобюджетных фильмов снялся: Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [50]:
answers['17'] = '2. Action'

In [51]:
data_17 = data_actors[data_actors['actor'] == 'Nicolas Cage']
data_17 = data_17.merge(data_genres_parsed, how='inner', on='imdb_id')
answer_17 = data_17.drop(['imdb_id','actor'], axis=1).sum().sort_values(ascending=False).head(1).index[0]


print('17. Nicolas Cage чаще всего снимался в фильмах жанра', answer_17)

17. Nicolas Cage чаще всего снимался в фильмах жанра Action


# 18. Самый убыточный фильм от Paramount Pictures

In [52]:
answers['18'] = '1. K-19: The Widowmaker'

In [53]:
data_18 = data_companies.merge(data[['imdb_id','profit', 'original_title']], how='outer', on='imdb_id')
data_Paramount_Pictures = data_18[data_18['company'] == 'Paramount Pictures']
data_18 = data_Paramount_Pictures[data_Paramount_Pictures['profit'] == data_Paramount_Pictures['profit'].min()]
answer_18 = data_18['original_title'].iloc[0]


print('18. Самым убыточным фильмом от Paramount Pictures оказался', answer_18)

18. Самым убыточным фильмом от Paramount Pictures оказался K-19: The Widowmaker


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [54]:
answers['19'] = '5. 2015'

In [55]:
pivot_19 = data.pivot_table(index='release_year', values='revenue',
                            aggfunc='sum', fill_value=0)
answer_19 = pivot_19.sort_values(by='revenue', ascending=False).iloc[0]


print('19. Самым успешным по суммарным кассовым сборам оказался', answer_19.name, 'год')

19. Самым успешным по суммарным кассовым сборам оказался 2015 год


# 20. Какой самый прибыльный год для студии Warner Bros?

По моему скромному мнению, корректно этот вопрос будет звучать так: Какой самый прибыльный год для студий Warner Bros?

In [56]:
answers['20'] = '1. 2014'

In [57]:
data_20 = data_companies.merge(data[['imdb_id','profit', 'release_year', 'original_title']],
                               how='outer', on='imdb_id')

data_20['Warner_Bros_company'] = data_20['company'].apply(lambda x: 1 if 'Warner Bros.' in x else 0)
data_Warner_Bros = data_20[data_20['Warner_Bros_company'] == 1]
pivot_20 = data_Warner_Bros.pivot_table(index = 'release_year', values = 'profit',
                                        aggfunc = 'sum',fill_value=0)
answer_20 = pivot_20.sort_values(by='profit', ascending=False)


print('20. Самым прибыльным годом для студий Warner Broc оказался', answer_20.iloc[0].name, 'год')

20. Самым прибыльным годом для студий Warner Broc оказался 2014 год


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [58]:
answers['21'] = '4. Сентябрь'

In [59]:
answer_21 = data_dates['month'].value_counts().sort_values(ascending=False).head(1)


print('21. За все годы вышло больше всего фильмов в', answer_21.index[0], 'месяце')

21. За все годы вышло больше всего фильмов в 9 месяце


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [60]:
answers['22'] = '2. 450'

In [61]:
movies_jun = data_dates['month'].value_counts().loc[6]
movies_jul = data_dates['month'].value_counts().loc[7]
movies_aug = data_dates['month'].value_counts().loc[8]
answer_22 = movies_jun + movies_jul + movies_aug


print('22. Летом вышло', answer_22, 'фильмов')

22. Летом вышло 450 фильмов


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [62]:
answers['23'] = '5. Peter Jackson'

In [63]:
data_23 = data_directors.merge(data_dates[['imdb_id', 'month']], on='imdb_id', how='inner')
data_23['winter'] = data_23['month'].apply(lambda x: 1 if x in [1,2,12] else 0)
data_winter = data_23[data_23.winter == 1]
answer_23 = data_winter['author'].value_counts().head(1)


print('23. Зима оказалась самым продуктивным временем года для', answer_23.index[0])

23. Зима оказалась самым продуктивным временем года для Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

Данный вопрос может трактоваться двумя разными способами. Рассмотрим оба случая.

ВАРИАНТ 1

Гипотеза: Ключевая величина - максимальная длина названия фильма среди всех в списке представленных

In [64]:
answers['24'] = '5. Four By Two Productions'

In [65]:
data_24 = data[['imdb_id', 'original_title']].copy(deep=True)
data_24['title_length'] = data_24['original_title'].str.len()
data_24 = data_24.merge(data_companies, on='imdb_id', how='inner')
answer_24 = data_24[data_24.title_length == data_24.title_length.max()]


print('24. Следующие студии дают самые длинные названия своим фильмам:')
print(answer_24['company'].tolist())
print('P.S. Ответ выбран в соответствии с представленными вариантами')

24. Следующие студии дают самые длинные названия своим фильмам:
['Twentieth Century Fox Film Corporation', 'Four By Two Productions']
P.S. Ответ выбран в соответствии с представленными вариантами


ВАРИАНТ 2

Гипотеза: Ключевая величина - средняя длина названия фильма для каждой студии

In [66]:
data_24 = data[['imdb_id', 'original_title']].copy(deep=True)
data_24['title_length'] = data_24['original_title'].apply(lambda x: len(x))
data_24 = data_24.merge(data_companies, on='imdb_id', how='inner')
data_24.sort_values(by='title_length', ascending=False)

pivot_24 = data_24.pivot_table(index='company', values='title_length',
                               aggfunc='sum', fill_value=0)

data_24 = data_companies['company'].value_counts()
data_24 = pd.concat([data_24, pivot_24], axis=1)

data_24['title_length_mean'] = data_24['title_length']/data_24['company']
answer_24 = data_24[data_24.title_length_mean == data_24.title_length_mean.max()]


print('24. Следующая студия даёт самые длинные названия своим фильмам:', answer_24.index[0])

24. Следующая студия даёт самые длинные названия своим фильмам: Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [67]:
answers['25'] = '3. Midnight Picture Show'

In [68]:
data_25 = data[['imdb_id', 'overview']].copy(deep=True)
data_25['overview_length'] = data_25['overview'].str.count(' ')
data_25 = data_25.merge(data_companies, on='imdb_id', how='inner')
pivot_25 = data_25.pivot_table(index='company', values='overview_length',
                               aggfunc='sum', fill_value=0)
data_25 = data_companies['company'].value_counts()
data_25 = pd.concat([data_25, pivot_25], axis=1)
data_25['overview_length_mean'] = data_25['overview_length']/data_25['company']


answer_25 = data_25[data_25.overview_length_mean == data_25.overview_length_mean.max()]
print('25. В среднем самые длинные описания по числу слов имеет студия', answer_25.index[0])

25. В среднем самые длинные описания по числу слов имеет студия Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [69]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [70]:
# Count 1% out of all movies
count_movies_1p = int(round(data['imdb_id'].nunique()/100, 0))
data_26 = data[['imdb_id','original_title','vote_average']]
answer_26 = data_26.sort_values(by='vote_average', ascending=False).head(count_movies_1p)


print('26. В 1 проценте лучших по рейтингу фильмов входят (в порядке убывания рейтинга):')
for movie in answer_26['original_title'].tolist():
    print(movie)

26. В 1 проценте лучших по рейтингу фильмов входят (в порядке убывания рейтинга):
The Dark Knight
Interstellar
The Imitation Game
Inside Out
Room
The Wolf of Wall Street
Gone Girl
12 Years a Slave
Guardians of the Galaxy
The Lord of the Rings: The Return of the King
Memento
Inception
The Pianist
The Grand Budapest Hotel
Her
Spotlight
Big Hero 6
The Fault in Our Stars
The Lord of the Rings: The Two Towers


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [71]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 1. ДОЛГИЙ

In [73]:
# Hypothesis: actors, who played only once or a few times could not be in the answer's list
# This parameter is noted as hyp and it's nature is pretty empirical
# Greater hyp leads to less evaluation time, but it should be too large.

hyp = 5

# Count movies to each actor indexed by imdb_id

data_27 = data_actors['actor'].value_counts()
print('Всего вариантов', len(data_27))
data_27 = data_27[data_27 > hyp]
print('Уменьшенное число вариантов', len(data_27))
data_27 = pd.DataFrame(data_27)
data_27 = data_27.reset_index()
data_27 = data_27.rename(columns={'actor':'count_movies','index':'actor'})
data_27_1 = data_actors.merge(data_27, how='inner', on='actor')


# Go thru the created DataFrame and count the common movies for each pair

actors_27 = data_27_1['actor'].unique()
data_27_1.drop('count_movies', axis=1)
cand = []
_n = len(actors_27)
for i, actor_x in enumerate(actors_27):
    for j, actor_y in enumerate(actors_27[i+1:]):
        movies_x = data_27_1[data_27_1['actor'] == actor_x]
        movies_y = data_27_1[data_27_1['actor'] == actor_y]
        n = len(movies_x.merge(movies_y, how='inner', on='imdb_id'))
        if n > 1:
            cand.append(np.array([[actor_x, actor_y], n], dtype=object))

answer_27 = np.array(cand, dtype=object)


print('27. Чаще всего снимались вместе следующие пары актёров:')
for pair in answer_27[answer_27[:,1] == answer_27[:,1].max()]:
    print(str(pair[0])[1:-1])

Всего вариантов 3406
Уменьшенное число вариантов 443
27. Чаще всего снимались вместе следующие пары актёров:
'Daniel Radcliffe', 'Emma Watson'
'Daniel Radcliffe', 'Rupert Grint'
'Emma Watson', 'Rupert Grint'


ВАРИАНТ 2. БЫСТРЕЕ ВАРИАНТА 1.

In [74]:
# Create a list of the DataFrames, where each element is corresponded to the unique imdb_id,
# and contain all combinations of actors' pairs.

tmp_list = []
for imdb_id in data['imdb_id']:
    data_27 = data_actors[data_actors['imdb_id'] ==  imdb_id]
    data_27 = data_27.sort_values(by='actor')
    if len(data_27) >=2:
        data_27_1 = pd.concat([data_27[i:i+1].merge(data_27[i+1:],
                                                    how='inner',
                                                    on='imdb_id') for i in range(len(data_27)-1)])
        tmp_list.append(data_27_1)

        
data_27 = pd.concat(tmp_list)
data_27 = data_27.groupby(["actor_x", "actor_y"]).size().sort_values(ascending=False)
answer_27 = data_27[data_27 == data_27.max()]


print('27. Чаще всего снимались вместе следующие пары актёров:')
for pair in answer_27.index:
    print(str(pair)[1:-1])

27. Чаще всего снимались вместе следующие пары актёров:
'Daniel Radcliffe', 'Rupert Grint'
'Emma Watson', 'Rupert Grint'
'Daniel Radcliffe', 'Emma Watson'


# Submission

In [75]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '2. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [76]:
# и убедиться что ни чего не пропустил)
len(answers)

27